# XHARPy Example: L-Alanin with GPAW
- Program: NoSpherA2/Orca
- B3LYP calculation
- 8 Angstroem cluster charges

In [ ]:
from xharpy import (shelxl_hkl2pd, cif2data, lst2constraint_dict, create_construction_instructions,
                    refine, write_cif, write_res, write_fcf, add_density_entries_from_fcf)
import os

In [ ]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    

### Loading Data

In [ ]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [ ]:
hkl = shelxl_hkl2pd('iam.hkl')
constraint_dict = lst2constraint_dict('iam.lst')

### Setting Options

In [ ]:
computation_dict = {
    'nosphera2_accuracy': 4,
    'basis_set': 'def2-TZVPP',
    'functional': 'B3LYP',
    'charge': 0,
    'multiplicity': 1,
    'n_cores': 2,
    'calc_folder': './calc',
    'orca_path': '/path/to/orcha',
    'nosphera2_path': '/path/to/NoSpherA2',
    'build_dict': {},
    'cutoff': 8.0
}

refinement_dict = {
    'f0j_source': 'nosphera2_orca',
    'reload_step': 1,
    'core': 'combine',
}

filename = computation_dict['functional'] + '_' + computation_dict['basis_set']

### Refinement

In [ ]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)

In [ ]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

### Writing Data to Disk

In [ ]:
write_cif(
    output_cif_path=os.path.join(output_folder, filename + '.cif'),
    cif_dataset=filename,
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '.fcf'),
    fcf_dataset=filename,
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '_6.fcf'),
    fcf_dataset=filename + '_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_res(
    out_res_path=os.path.join(output_folder, filename + '_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [ ]:
add_density_entries_from_fcf(
    os.path.join(output_folder, filename + '.cif'),
    os.path.join(output_folder, filename + '_6.fcf')
)